In [1]:
from imports import *
from classes import *

In [30]:
model = MaskablePPO.load("./models/Skyjo_champion_19")
model1 = MaskablePPO.load("./models/Skyjo_v0.2.8")

In [31]:
def action_state(joueur1, joueur2, model, phase, prec_action, revealedcard, type_model):
    inputs = np.array([13] * 25 + [0] * 1, dtype=np.float32)
    if phase:
        action_mask = np.array([0] * 2 + [1] * 12 + [0] * 12)
        if not prec_action:
            action_mask[14:26] = [1 * (joueur_1[i] == 13) for i in range(0, 12)]
        inputs[25] = 1
    else:
        action_mask = np.array([1] * 2 + [0] * 24)
        inputs[25] = 0
    inputs[0], inputs[1:13], inputs[13:25] = revealedcard, joueur1, joueur2
    if type_model == "ancien":
        return model.predict(inputs, action_masks=action_mask, deterministic=True)[0]
    else:
        revealed = [0] + [1 * (x != 13) for x in joueur_1 + joueur_2] + [0]
        values = [inputs[0]] + [x * (x != 13) for x in inputs[1:25]] + [inputs[25]]
        obs = {"observations": values, "revealed": revealed}
        return model.predict(obs, action_masks=action_mask, deterministic=True)[0]

def print_action(action):
    if action == 0:
        print("Prendre carte cachée")
    elif action == 1:
        print("Prendre carte visible")
    elif action <= 13:
        print(f"Échanger carte n°{action - 1}")
    else:
        print(f"Révéler carte n°{action - 13}")

In [32]:
joueur_1 = [13, 13, 13, 13,
            13, 13, 13, 13,
            13, 13, 13, 13]
joueur_2 = [13, 13, 13, 13,
            13, 13, 13, 13,
            13, 13, 13, 13]

In [99]:
phase, prec_action, revealed_card = 1, 0, 2

joueur_1 = [5, 0, 1, -1,
            1, -1, 13, -2,
            0, -1, -2, 2]
joueur_2 = [0, -2, 0, 0,
            0, -1, 10, 2,
            0, 1, 13, 2]

action = action_state(joueur_1, joueur_2, model, phase, prec_action, revealed_card, "nouveau")
action1 = action_state(joueur_1, joueur_2, model1, phase, prec_action, revealed_card, "nouveau")
print('---------------- Nouveau moteur -------------------')
print_action(action)
print('---------------- Vieux moteur -------------------')
print_action(action1)

---------------- Nouveau moteur -------------------
Échanger carte n°7
---------------- Vieux moteur -------------------
Échanger carte n°1


In [29]:
for i in range(0, 21):
    model = MaskablePPO.load(f"./models/Skyjo_champion_{i}")
    results = []
    for _ in range(0, 3000):
        results.append(model_fight(model, "heuristic"))
    
    print(f"Modèle {i+1} vs heuristic: {round(np.mean(results), 2)}")

Modèle 1 vs heuristic: -24.07
Modèle 2 vs heuristic: -9.84
Modèle 3 vs heuristic: -0.66
Modèle 4 vs heuristic: 1.74
Modèle 5 vs heuristic: 1.56
Modèle 6 vs heuristic: 5.57
Modèle 7 vs heuristic: 5.57
Modèle 8 vs heuristic: 4.56
Modèle 9 vs heuristic: 4.45
Modèle 10 vs heuristic: 6.28
Modèle 11 vs heuristic: 6.52
Modèle 12 vs heuristic: 6.12
Modèle 13 vs heuristic: 5.46
Modèle 14 vs heuristic: 4.22
Modèle 15 vs heuristic: 5.06
Modèle 16 vs heuristic: 4.76
Modèle 17 vs heuristic: 4.97
Modèle 18 vs heuristic: 4.84
Modèle 19 vs heuristic: 6.68
Modèle 20 vs heuristic: 7.5


FileNotFoundError: [Errno 2] No such file or directory: 'models\\Skyjo_champion_20.zip'